Import statements

In [62]:
import numpy as np
import pandas as pd
import os
import glob
from ast import literal_eval

Load the Data adequately

In [7]:
# the name of our directory
dname = os.path.dirname(os.getcwd()) + "/data_loading/data_v2_quotes/"
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts/data_loading/data_v2_quotes/'

In [50]:
# load the data
df = pd.read_csv(dname + "full_data.csv")
# drop redundant columns
df.drop(columns=["trial_bonus", "block_bonus", "resp_indices", "resp_indices_values", "resp_timestamp", "resp_reactiontime"], inplace=True)
df.head(3)
df.shape

(1280, 16)

In [105]:
# find the subjects
sujs = df['id'].unique()
# select all rows from first block
df_block0 = df.loc[df['expt_block'] == 0] # df.loc[:, df.loc['expt_block'] == 0]
df_block0.shape
# select all rows for our first category
df_block0_cat01 = df_block0.loc[df_block0['vbxi_category'].isin([0, 1])]
df_block0_cat01.head(3)



[0 1 2 3 4 5 6 7 8 9]


,Unnamed: 0,id,expt_index,expt_trial,expt_block,trial_att,vbxi_category,exemplar,trialProperties,trialPropShuff,property_order,resp_correct_array,resp_correct_array_unshuf,prop_responses,groundTruthCorrect,groundTruthCorrect_unshuf
3,3,0,3,3,0,1,0,13,"[1, 1, 0, 1, 0, 0, 0]","[[1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1]]","[[0, 6, 1, 5, 3, 4, 2], [1, 0, 6, 2, 5, 4, 3]]","[[None, False, None, False, True, None, None],...","[[None, None, None, True, None, False, False],...","[[None, 1, None, 1, 1, None, None], [1, 1, Non...","[[0, 0, 0, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]]"
5,5,0,5,5,0,1,1,9,"[1, 1, 0, 1, 0, 0, 0]","[[0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 1]]","[[2, 3, 0, 4, 6, 1, 5], [2, 6, 1, 5, 4, 0, 3]]","[[None, None, None, False, False, True, None],...","[[None, True, None, None, False, None, False],...","[[None, None, None, 1, 1, 1, None], [None, Non...","[[1, 0, 0, 0, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1]]"
7,7,0,7,7,0,0,0,10,"[1, 1, 0, 1, 0, 0, 0]","[[1, 0, 0, 1, 0, 0, 1]]","[[0, 2, 4, 3, 5, 6, 1]]","[[True, None, None, True, None, None, True]]","[[True, True, None, True, None, None, None]]","[[1, None, None, 1, None, None, 1]]","[[1, 1, 1, 1, 1, 1, 1]]","[[1, 1, 1, 1, 1, 1, 1]]"


In [104]:
# get the average response vector in the first block for the first two categories
vec_bl0_cat01 = df_block0_cat01["resp_correct_array_unshuf"]
vec_bl0_cat01 = vec_bl0_cat01.apply(lambda x: np.asarray(literal_eval(x)[0]))
vec_bl0_cat01 = vec_bl0_cat01.apply(lambda x: np.select([x == True, x == False],[1, 1]))
vec_bl0_cat01 =  vec_bl0_cat01.sum(axis=0)/vec_bl0_cat01.shape[0]
print(vec_bl0_cat01)

[0.75       0.67857143 0.21428571 0.58928571 0.28571429 0.26785714
 0.21428571]


In [ ]:
def input_output_matrix(df):
    """function to create the input output matrix"""
    
    blocks = df['expt_block'].unique()
    categs = df['vbxi_category'].unique()

    in_out_matrix = np.full((8, len(categs)/2, len(blocks)), np.nan)

    for block in blocks:
        df_block = df.loc[df['expt_block'] == block]
        for categ in categs[::2]:

            df_block_categ = [df_block['vbxi_category'].isin([categ, categ + 1])]
            df_block_categ = df_block_categ["resp_correct_array_unshuf"]
            df_block_categ = df_block_categ.apply(lambda x: np.asarray(literal_eval(x)[0]))
            df_block_categ = df_block_categ.apply(lambda x: np.select([x == True, x == False],[1, 1]))

            array_block_categ = df_block_categ.sum(axis=0)/df_block_categ.shape[0]
            in_out_matrix[:, ]